In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digits/digits updated/digits updated/7/img008-00675.png
/kaggle/input/digits/digits updated/digits updated/7/img008-00327.png
/kaggle/input/digits/digits updated/digits updated/7/img008-00721.png
/kaggle/input/digits/digits updated/digits updated/7/img008-00137.png
/kaggle/input/digits/digits updated/digits updated/7/img008-00450.png
/kaggle/input/digits/digits updated/digits updated/7/img008-00002.png
/kaggle/input/digits/digits updated/digits updated/7/img008-00428.png
/kaggle/input/digits/digits updated/digits updated/7/img008-00776.png
/kaggle/input/digits/digits updated/digits updated/7/img008-00896.png
/kaggle/input/digits/digits updated/digits updated/7/img008-00188.png
/kaggle/input/digits/digits updated/digits updated/7/img008-00021.png
/kaggle/input/digits/digits updated/digits updated/7/img008-00080.png
/kaggle/input/digits/digits updated/digits updated/7/img008-00382.png
/kaggle/input/digits/digits updated/digits updated/7/img008-00459.png
/kaggle/input/digits

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.models import Sequential
from keras.src.legacy.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
import os
import matplotlib.pyplot as plt
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

2024-03-19 10:05:16.797227: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-19 10:05:16.797335: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-19 10:05:16.929266: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
train_data_dir='/kaggle/input/digits/Digits/Digits/'
#val_data_dir='/content/drive/MyDrive/sudoku_digits/data_v2/validation_data'

In [4]:
mkdir /kaggle/working/checkpoints

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    shear_range=0.3,
    zoom_range=0.3,
    width_shift_range=0.4,
    height_shift_range=0.4,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2)


val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)


In [6]:
train_generator = train_datagen.flow_from_directory(
 train_data_dir,
 #color_mode='grayscale',
 target_size=(255,255),
 batch_size=4,
 class_mode='categorical',
 subset='training',
 shuffle=True)


validation_generator = val_datagen.flow_from_directory(
 train_data_dir,
 #color_mode='grayscale',
 target_size=(255,255),
 batch_size=4,
 class_mode='categorical',
 subset='validation',
 shuffle=True)

Found 8130 images belonging to 10 classes.
Found 2030 images belonging to 10 classes.


In [7]:
import tensorflow as tf

tf.__version__

'2.15.0'

In [8]:
checkpoint = ModelCheckpoint('/kaggle/working/checkpoint/Sudoku_Model.keras',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=7,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=5,
                              verbose=1,
                              min_delta=0.0001)


callbacks = [earlystop,checkpoint,reduce_lr]

In [9]:
from keras.applications.inception_v3 import InceptionV3

from tensorflow.keras import regularizers

In [10]:
conv_base_Inception = InceptionV3(
    weights='imagenet',
    include_top = False,
    input_shape=(255, 255, 3)
)

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [11]:
conv_base_Inception.summary()

Model: "inception_v3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 255, 255,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 127, 127,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 127, 127,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 127, 127,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 125, 125,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 125, 125,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 125, 125,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 125, 125,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 125, 125,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 125, 125,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 62, 62,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 62, 62,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 62, 62,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 62, 62,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 60, 60,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 60,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 60, 60,    │          0 │ batch_normalizat

 Total params: 21,802,784 (83.17 MB)

 Trainable params: 21,768,352 (83.04 MB)

 Non-trainable params: 34,432 (134.50 KB)

In [12]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization, GlobalAveragePooling2D
from keras.models import Model


x = conv_base_Inception.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
x = Dense(562, activation='relu')(x)
x = Dense(200, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(10, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=conv_base_Inception.input, outputs=predictions)

In [13]:
model.summary

<bound method Model.summary of <Functional name=functional_1, built=True>>

In [14]:
model.compile(optimizer=Adam(1e-5) , loss='categorical_crossentropy', metrics=['accuracy'])


In [15]:
history_Incep = model.fit(train_generator,epochs=10, validation_data=validation_generator, callbacks=callbacks)

Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


   1/2033 ━━━━━━━━━━━━━━━━━━━━ 45:38:50 81s/step - accuracy: 0.2500 - loss: 2.1291

I0000 00:00:1710842814.427172      91 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1710842814.530063      91 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


 785/2033 ━━━━━━━━━━━━━━━━━━━━ 2:24 116ms/step - accuracy: 0.2832 - loss: 2.0666

W0000 00:00:1710842905.270121      90 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


2033/2033 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.4624 - loss: 1.6308

W0000 00:00:1710842994.716982      90 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1710843007.065097      92 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update



Epoch 1: val_loss improved from inf to 0.16317, saving model to /kaggle/working/checkpoint/Sudoku_Model.keras
2033/2033 ━━━━━━━━━━━━━━━━━━━━ 276s 96ms/step - accuracy: 0.4625 - loss: 1.6306 - val_accuracy: 0.9700 - val_loss: 0.1632 - learning_rate: 1.0000e-05
Epoch 2/10
2033/2033 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.8629 - loss: 0.4438
Epoch 2: val_loss improved from 0.16317 to 0.07635, saving model to /kaggle/working/checkpoint/Sudoku_Model.keras
2033/2033 ━━━━━━━━━━━━━━━━━━━━ 139s 68ms/step - accuracy: 0.8629 - loss: 0.4438 - val_accuracy: 0.9847 - val_loss: 0.0764 - learning_rate: 1.0000e-05
Epoch 3/10
2033/2033 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.8959 - loss: 0.3311
Epoch 3: val_loss did not improve from 0.07635
2033/2033 ━━━━━━━━━━━━━━━━━━━━ 138s 68ms/step - accuracy: 0.8959 - loss: 0.3311 - val_accuracy: 0.9207 - val_loss: 0.2791 - learning_rate: 1.0000e-05
Epoch 4/10
2033/2033 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.9117 - loss: 0.2754
Epoch 4: va

In [16]:
cd /kaggle/working/checkpoints

/kaggle/working/checkpoints


In [17]:
!zip sudoku_model.zip Sudoku_Model.keras

	zip warning: name not matched: Sudoku_Model.keras

zip error: Nothing to do! (sudoku_model.zip)


In [18]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [19]:
import keras
print(keras.__version__)

3.0.5
